In [11]:
library(MASS)
library(MLmetrics)

### Tools

In [12]:
class_table <- function(a,b){
      sen <- Sensitivity(a,b)
      recal <- Recall(a,b)
      preci <- Precision(a,b)
      spec <- Specificity(a,b)
      f1score <- F1_Score(a,b)
      cat('Sensitivity:', sen,'\nRecall:', recal,'\nPrecision:',preci,'\nSpecificity:', spec ,'\nF1_Score:',f1score)
      table <- c(sen, recal, preci, spec, f1score)
}

# Q1

In [13]:
possum = read.csv('/Users/gawain/Desktop/3.UBC/Block4/Data-572-Supervised learning/Assignment/Assignment1/possum.csv')
head(possum)

,Pop,hdlngth,skullw,totlngth,taill
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Vic,94.1,60.4,89.0,36.0
2,Vic,92.5,57.6,91.5,36.5
3,Vic,94.0,60.0,95.5,39.0
4,Vic,93.2,57.1,92.0,38.0
5,Vic,91.5,56.3,85.5,36.0
6,Vic,93.1,54.8,90.5,35.5


In [14]:
# Use Pop as the response, with the remainder of the variables as predictors.
# possum$Pop <- as.factor(possum$Pop)
possum = read.csv('/Users/gawain/Desktop/3.UBC/Block4/Data-572-Supervised learning/Assignment/Assignment1/possum.csv')

# possum$Pop <- as.numeric(as.factor(possum$Pop))
possum[possum == 'Vic'] <- 1
possum[possum == 'other'] <- 0
possum$Pop <- as.numeric(possum$Pop)

# possum$Pop <- possum$Pop-1
head(possum)
cat('unique value in possum$Pop:', unique(possum$Pop))

,Pop,hdlngth,skullw,totlngth,taill
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,94.1,60.4,89.0,36.0
2,1,92.5,57.6,91.5,36.5
3,1,94.0,60.0,95.5,39.0
4,1,93.2,57.1,92.0,38.0
5,1,91.5,56.3,85.5,36.0
6,1,93.1,54.8,90.5,35.5


unique value in possum$Pop: 1 0

### Part a

Fit linear discriminant analysis. 

Provide a classification table and other metrics we have covered in lecture/lab.

In [15]:
Pop_lda <- lda(Pop~., data=possum, CV=TRUE)
table(possum$Pop, Pop_lda$class)

   
     0  1
  0 48 10
  1  9 37

In [16]:
lda_info <- class_table(possum$Pop, Pop_lda$class)
logloss_lda <- LogLoss(Pop_lda$posterior[,2], as.numeric(possum$Pop))
cat('\nLog Loss:', logloss_lda)
lda_info <- append(lda_info, logloss_lda)

Sensitivity: 0.8275862 
Recall: 0.8275862 
Precision: 0.8421053 
Specificity: 0.8043478 
F1_Score: 0.8347826
Log Loss: 0.4215897

In [17]:
lda_info

[1] 0.8275862 0.8275862 0.8421053 0.8043478 0.8347826 0.4215897

### Part b

Fit quadratic discriminant analysis. Provide a classification table and other metrics we have covered in lecture/lab.

In [18]:
Pop_qda <- qda(Pop~., data=possum, CV=TRUE)
table(possum$Pop, Pop_qda$class)

   
     0  1
  0 51  7
  1  9 37

In [19]:
qda_info <- class_table(possum$Pop, Pop_qda$class)
logloss_qda <- LogLoss(Pop_qda$posterior[,2], possum$Pop)
cat('\nLog Loss:', logloss_qda)
qda_info <- append(qda_info, logloss_qda)

Sensitivity: 0.8793103 
Recall: 0.8793103 
Precision: 0.85 
Specificity: 0.8043478 
F1_Score: 0.8644068
Log Loss: 0.6214646

### Part c

Fit k-nearest neighbours (K = 3). Provide a classification table and other metrics we have covered in lecture/lab.

In [20]:
library(class)
knn3 <- knn.cv(possum[,-1], possum$Pop, k=3, prob=TRUE)
table(possum$Pop, knn3)

   knn3
     0  1
  0 49  9
  1 12 34

In [21]:
knn_info <- class_table(possum$Pop, knn3)

Sensitivity: 0.8448276 
Recall: 0.8448276 
Precision: 0.8032787 
Specificity: 0.7391304 
F1_Score: 0.8235294

In [22]:
probs <- attr(knn3, "prob")
missedprobs <- 1-probs[possum$Pop!=knn3]
missedprobs[missedprobs==0] <- 1e-3
probs[probs==0] <- 1e-3
logloss_knn <- (sum(-log(probs[possum$Pop==knn3])) + sum(-log(missedprobs)))/length(possum$Pop)
cat('log Loss:', logloss_knn)
knn_info <- append(knn_info, logloss_knn)

log Loss: 0.6739391

### Part d

Which of the above models would you suggest is ‘best’ for this data? Why?

In [23]:
DF1 <- as.data.frame(lda_info)    # Convert array to data frame
DF2 <- as.data.frame(qda_info) 
DF3 <- as.data.frame(knn_info) 
DF4 <- cbind(DF1, DF2, DF3)
DF4$index <- c('Sensitivity', 'Recall', 'Precision', 'Specificity','F1-Score', 'Log Loss')
DF <- DF4[, c("index", "lda_info", "qda_info", "knn_info")]
DF

index,lda_info,qda_info,knn_info
<chr>,<dbl>,<dbl>,<dbl>
Sensitivity,0.8275862,0.8793103,0.8448276
Recall,0.8275862,0.8793103,0.8448276
Precision,0.8421053,0.8500000,0.8032787
Specificity,0.8043478,0.8043478,0.7391304
F1-Score,0.8347826,0.8644068,0.8235294
Log Loss,0.4215897,0.6214646,0.6739391


Based on the result from above dataframe, we expect all evaluation value except **Log Loss** as big as possible, and expect Log Loss as small as possible.

Thus, I think QDA is the 'best' model, as it has the highest Specificity, Recall, Precision, and F1-score, and smallest Log Loss

# Q2

In [24]:
okcupid = read.csv('/Users/gawain/Desktop/3.UBC/Block4/Data-572-Supervised learning/Assignment/Assignment1/okcupidprofiles.csv')
head(okcupid)

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,⋯,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,22,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75,-1,transportation,⋯,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
2,35,average,mostly other,often,sometimes,working on space camp,white,70,80000,hospitality / travel,⋯,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (poorly)",single
3,38,thin,anything,socially,,graduated from masters program,,68,-1,,⋯,"san francisco, california",,straight,has cats,,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available
4,23,thin,vegetarian,socially,,working on college/university,white,71,20000,student,⋯,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,,m,pisces,no,"english, german (poorly)",single
5,29,athletic,,socially,never,graduated from college/university,"asian, black, other",66,-1,artistic / musical / writer,⋯,"san francisco, california",,straight,likes dogs and likes cats,,m,aquarius,no,english,single
6,29,average,mostly anything,socially,,graduated from college/university,white,67,-1,computer / hardware / software,⋯,"san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes cats,atheism,m,taurus,no,"english (fluently), chinese (okay)",single


In [25]:
# install.packages("arules")

In [26]:
library(arules)

Loading required package: Matrix


Attaching package: 'arules'


The following objects are masked from 'package:base':

    abbreviate, write




In [27]:
asurv <- apriori(okcupid)

Warning message:
"Column(s) 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21 not logical or factor. Applying default discretization (see '? discretizeDF')."


Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.8    0.1    1 none FALSE            TRUE       5     0.1      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 6074 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[39500 item(s), 60743 transaction(s)] done [0.45s].
sorting and recoding items ... [36 item(s)] done [0.02s].
creating transaction tree ... done [0.04s].
checking subsets of size 1 2 3 4 5 6 7 done [0.38s].
writing ... [3656 rule(s)] done [0.00s].
creating S4 object  ... done [0.03s].


In [28]:
summary(asurv)

set of 3656 rules

rule length distribution (lhs + rhs):sizes
   1    2    3    4    5    6    7 
   2   79  489 1236 1260  537   53 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   4.000   5.000   4.503   5.000   7.000 

summary of quality measures:
    support         confidence        coverage           lift       
 Min.   :0.1000   Min.   :0.8002   Min.   :0.1035   Min.   :0.9096  
 1st Qu.:0.1136   1st Qu.:0.8518   1st Qu.:0.1279   1st Qu.:1.0215  
 Median :0.1359   Median :0.8826   Median :0.1522   Median :1.0389  
 Mean   :0.1635   Mean   :0.8901   Mean   :0.1846   Mean   :1.0496  
 3rd Qu.:0.1812   3rd Qu.:0.9387   3rd Qu.:0.2044   3rd Qu.:1.0593  
 Max.   :0.9169   Max.   :0.9747   Max.   :1.0000   Max.   :1.3816  
     count      
 Min.   : 6075  
 1st Qu.: 6902  
 Median : 8254  
 Mean   : 9931  
 3rd Qu.:11005  
 Max.   :55695  

mining info:
    data ntransactions support confidence                    call
 okcupid         60743     0.1        0.8 apriori(data 

In [29]:
inspect(asurv[1:10])

     lhs                                    rhs                      support confidence  coverage      lift count
[1]  {}                                  => {orientation=straight} 0.8495300  0.8495300 1.0000000 1.0000000 51603
[2]  {}                                  => {status=single}        0.9168958  0.9168958 1.0000000 1.0000000 55695
[3]  {location=oakland, california}      => {status=single}        0.1072058  0.9026892 0.1187627 0.9845058  6512
[4]  {pets=likes dogs}                   => {orientation=straight} 0.1056747  0.8885659 0.1189273 1.0459500  6419
[5]  {pets=likes dogs}                   => {status=single}        0.1138897  0.9576412 0.1189273 1.0444384  6918
[6]  {education=}                        => {status=single}        0.1018554  0.8340523 0.1221211 0.9096479  6187
[7]  {offspring=doesn&rsquo;t have kids} => {income=-1}            0.1042754  0.8379415 0.1244423 1.0507655  6334
[8]  {offspring=doesn&rsquo;t have kids} => {orientation=straight} 0.1150421  0.9244609 

Support: The top rule with the highest support is {} => {status=single} with a support value of 0.9168958, indicating that 91.69% of the transactions contain the status=single item.

Confidence: The top rule with the highest confidence is {pets=likes dogs} => {status=single} with a confidence value of 0.9576412, indicating that 95.76% of the transactions that contain pets=likes dogs also contain status=single.

Lift: The top rule with the highest lift is {offspring=doesn't have kids} => {orientation=straight} with a lift value of 1.0882028, indicating that there is a stronger association between {offspring=doesn't have kids} and {orientation=straight} than would be expected if these items were independent.

Overall, it seems that the single status and straight orientation are highly associated with each other, and that having pets, education, and offspring are more associated with other variables like income and orientation than status.

### There is some detailed info for support, confidence and lift

In [30]:
inspect(sort(asurv, by="support")[1:10])

     lhs                                  rhs                    support  
[1]  {}                                => {status=single}        0.9168958
[2]  {}                                => {orientation=straight} 0.8495300
[3]  {orientation=straight}            => {status=single}        0.8018702
[4]  {status=single}                   => {orientation=straight} 0.8018702
[5]  {income=-1}                       => {status=single}        0.7446290
[6]  {status=single}                   => {income=-1}            0.7446290
[7]  {income=-1}                       => {orientation=straight} 0.6838154
[8]  {orientation=straight}            => {income=-1}            0.6838154
[9]  {smokes=no}                       => {status=single}        0.6710074
[10] {income=-1, orientation=straight} => {status=single}        0.6479430
     confidence coverage  lift     count
[1]  0.9168958  1.0000000 1.000000 55695
[2]  0.8495300  1.0000000 1.000000 51603
[3]  0.9438986  0.8495300 1.029450 48708
[4]  0.8745

In [31]:
inspect(sort(asurv, by="confidence")[1:10])

     lhs                                     rhs               support confidence  coverage     lift count
[1]  {offspring=doesn&rsquo;t have kids,                                                                  
      orientation=straight}               => {status=single} 0.1121281  0.9746709 0.1150421 1.063012  6811
[2]  {drinks=socially,                                                                                    
      drugs=never,                                                                                        
      income=-1,                                                                                          
      orientation=straight,                                                                               
      religion=}                          => {status=single} 0.1086545  0.9737386 0.1115849 1.061995  6600
[3]  {diet=mostly anything,                                                                               
      drugs=never,                   

In [32]:
inspect(sort(asurv, by="lift")[1:10])

     lhs                                               rhs           support confidence  coverage     lift count
[1]  {body_type=athletic,                                                                                       
      orientation=straight}                         => {sex=m}     0.1426831  0.8148740 0.1750984 1.381619  8667
[2]  {body_type=athletic,                                                                                       
      orientation=straight,                                                                                     
      status=single}                                => {sex=m}     0.1372998  0.8128655 0.1689084 1.378214  8340
[3]  {body_type=athletic,                                                                                       
      drinks=socially,                                                                                          
      orientation=straight}                         => {sex=m}     0.1048681  0.8081705 0.129759

The summary of the association rules shows that there are 3656 rules in total, with the majority of the rules having 2, 3 or 4 items in their left-hand side and right-hand side. 

The minimum, first quartile, median, mean, third quartile and maximum values for support, confidence, coverage, and lift are also given. 

The top 10 rules sorted by support, confidence, and lift are also shown. 
The support measure represents the fraction of transactions that contain a rule.
The confidence measure represents the fraction of transactions that contain the left-hand side of the rule that also contain the right-hand side of the rule, coverage represents the fraction of transactions that contain the right-hand side of the rule.
The lift represents the ratio of the confidence to the expected confidence if the left-hand side and the right-hand side were independent.

### adujst AR

In [33]:
# adjust thresholds
asurv_adjust <- apriori(okcupid, parameter=list(supp=0.04, conf=.8))

Warning message:
"Column(s) 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21 not logical or factor. Applying default discretization (see '? discretizeDF')."


Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.8    0.1    1 none FALSE            TRUE       5    0.04      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 2429 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[39500 item(s), 60743 transaction(s)] done [0.40s].
sorting and recoding items ... [92 item(s)] done [0.02s].
creating transaction tree ... done [0.03s].
checking subsets of size 1 2 3 4 5 6 7 8 9 done [0.93s].
writing ... [25375 rule(s)] done [0.01s].
creating S4 object  ... done [0.05s].


In [34]:
summary(asurv_adjust)

set of 25375 rules

rule length distribution (lhs + rhs):sizes
   1    2    3    4    5    6    7    8    9 
   2  215 1598 4712 7742 7287 3230  582    7 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   4.000   5.000   5.274   6.000   9.000 

summary of quality measures:
    support          confidence        coverage            lift       
 Min.   :0.04000   Min.   :0.8000   Min.   :0.04083   Min.   :0.8761  
 1st Qu.:0.04601   1st Qu.:0.8570   1st Qu.:0.05155   1st Qu.:1.0262  
 Median :0.05615   Median :0.8909   Median :0.06285   Median :1.0467  
 Mean   :0.07247   Mean   :0.8950   Mean   :0.08137   Mean   :1.0690  
 3rd Qu.:0.07779   3rd Qu.:0.9421   3rd Qu.:0.08755   3rd Qu.:1.0842  
 Max.   :0.91690   Max.   :0.9910   Max.   :1.00000   Max.   :2.5916  
     count      
 Min.   : 2430  
 1st Qu.: 2795  
 Median : 3411  
 Mean   : 4402  
 3rd Qu.: 4725  
 Max.   :55695  

mining info:
    data ntransactions support confidence
 okcupid         60743    0.04        0.8
  

In [35]:
asurv_adjust <- apriori(okcupid, parameter=list(supp=0.04, conf=.8))
inspect(asurv_adjust[1:10])

Warning message:
"Column(s) 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21 not logical or factor. Applying default discretization (see '? discretizeDF')."


Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.8    0.1    1 none FALSE            TRUE       5    0.04      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 2429 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[39500 item(s), 60743 transaction(s)] done [0.40s].
sorting and recoding items ... [92 item(s)] done [0.02s].
creating transaction tree ... done [0.03s].
checking subsets of size 1 2 3 4 5 6 7 8 9 done [0.93s].
writing ... [25375 rule(s)] done [0.01s].
creating S4 object  ... done [0.04s].
     lhs                       rhs                    support    confidence
[1]  {}                     => {orientation=straight} 0.84952999 0.8495300 
[2]  {}                     => {status=single}        0.91689577 0.9168958 
[3]  {height=74}            => {sex=m}   

Based on the top 10 rules, it appears that the most frequent association is still between single status and straight orientation, and that there is also a high association between height, sex, and orientation. These rules have high support and confidence values indicating that they are frequent and trustworthy rules.

Additionally, it can be seen that the height and sex have a strong association as well. For example, height=74 and sex=m have a high support, confidence, and lift values.

It also appears that, with the new support and confidence, the rules are more specific with less generic rules like {} => {orientation=straight}, and that they are still consistent with the first analysis.

It's worth noting that by increasing the support and confidence threshold, the number of rules generated will decrease and the rules that remain will be more specific and have a higher likelihood of being true.

### There is some detailed info for support, confidence and lift

In [36]:
inspect(sort(asurv_adjust, by="support")[1:10])

     lhs                                  rhs                    support  
[1]  {}                                => {status=single}        0.9168958
[2]  {}                                => {orientation=straight} 0.8495300
[3]  {orientation=straight}            => {status=single}        0.8018702
[4]  {status=single}                   => {orientation=straight} 0.8018702
[5]  {income=-1}                       => {status=single}        0.7446290
[6]  {status=single}                   => {income=-1}            0.7446290
[7]  {income=-1}                       => {orientation=straight} 0.6838154
[8]  {orientation=straight}            => {income=-1}            0.6838154
[9]  {smokes=no}                       => {status=single}        0.6710074
[10] {income=-1, orientation=straight} => {status=single}        0.6479430
     confidence coverage  lift     count
[1]  0.9168958  1.0000000 1.000000 55695
[2]  0.8495300  1.0000000 1.000000 51603
[3]  0.9438986  0.8495300 1.029450 48708
[4]  0.8745

In [37]:
inspect(sort(asurv_adjust, by="confidence")[1:10])


     lhs                                                     rhs                support confidence   coverage     lift count
[1]  {height=74}                                          => {sex=m}         0.04155211  0.9909698 0.04193076 1.680190  2524
[2]  {offspring=doesn&rsquo;t have kids,                                                                                    
      orientation=straight,                                                                                                 
      religion=}                                          => {status=single} 0.04056434  0.9871795 0.04109115 1.076654  2464
[3]  {drinks=socially,                                                                                                      
      drugs=never,                                                                                                          
      income=-1,                                                                                                            


In [38]:
inspect(sort(asurv_adjust, by="lift")[1:10])

     lhs                        rhs            support confidence   coverage     lift count
[1]  {job=,                                                                                
      offspring=,                                                                          
      religion=,                                                                           
      sign=}                 => {pets=}     0.04189783  0.8839875 0.04739641 2.591633  2545
[2]  {education=,                                                                          
      offspring=,                                                                          
      sign=}                 => {pets=}     0.04038325  0.8732645 0.04624401 2.560196  2453
[3]  {job=,                                                                                
      religion=,                                                                           
      sign=}                 => {pets=}     0.04456481  0.8531358 0.05223647 2.5

# Q3

<img src='/Users/gawain/Desktop/3.UBC/Block4/Data-572-Supervised learning/Assignment/Assignment1/Q3.jpeg'>

# Q4

<img src='/Users/gawain/Desktop/3.UBC/Block4/Data-572-Supervised learning/Assignment/Assignment1/Q4.jpeg'>